In [ ]:
sample_in1 = """
#############
#...........#
###B#C#B#D###
  #A#D#C#A#
  #########
"""

my_in = """
#############
#...........#
###C#C#B#D###
  #D#A#B#A#
  #########
"""
PART0 = False
PART1 = True
PART2 = True
"""Graph:

1 - 2 - X - 3 - X - 4 - X - 5 - X - 6 - 7
        |       |       |       |
      Room1   Room2   Room3   Room4
        |       |       |       |
      Room1b  Room2b  Room3b  Room4b

can't stop at X spaces, 
"""
_G = {  # Valid "edges" and their cost
    (1, 2): 1, (6, 7): 1,
    (11, 21): 1, (12, 22): 1, (13, 23): 1, (14, 24): 1,
    (2, 11): 2, (3, 11): 2, (2, 3): 2, (3, 12): 2, (4, 12): 2, (3, 4): 2,
    (4, 13): 2, (5, 13): 2, (4, 5): 2, (5, 14): 2, (6, 14): 2, (5, 6): 2,
}

_G2 = { # Addicional "special" edges
  (1, 11): (3, [2]), (1, 12): (5, [2, 3]), (1, 13): (7, [2, 3, 4]), (1, 14): (9, [2, 3, 4, 5]),
  (2, 12): (4, [3]), (2, 13): (6, [3, 4]), (2, 14): (8, [3, 4, 5]),
  (3, 13): (4, [4]), (3, 14): (6, [4, 5]),
  (4, 11): (4, [3]), (4, 14): (4, [5]),
  (5, 12): (4, [4]), (5, 11): (6, [4, 3]),
  (6, 13): (4, [5]), (6, 12): (6, [5, 4]), (6, 11): (8, [5, 4, 3]),
  (7, 14): (3, [6]), (7, 13): (5, [6, 5]), (7, 12): (7, [6, 5, 4]), (7, 11): (9, [6, 5, 4, 3]),
  }

G = {edge: cost for _edge, cost in _G.items()
     for edge in [_edge, (_edge[1], _edge[0])]}
G2 = {edge: val for _edge, val in _G2.items()
     for edge in [_edge, (_edge[1], _edge[0])]}
for edge, cost in G.items():
  G2[edge] = (cost, [])


MOVE_COST = {
    'A': 1,
    'B': 10,
    'C': 100,
    'D': 1000,
}
print(G)


In [ ]:
import re


def get_input(s):
    lines = s.strip().split('\n')
    first_line = re.search('#([ABCD])#([ABCD])#([ABCD])#([ABCD])#', lines[2])
    second_line = re.search('#([ABCD])#([ABCD])#([ABCD])#([ABCD])#', lines[3])
    fl = first_line.groups()
    sl = second_line.groups()
    rooms = zip(fl, sl)
    return tuple(rooms)


print(get_input(sample_in1))
print(get_input(my_in))


In [ ]:
def prob1(s):
    # We want AA in Room1, BB in room2, CC in room3 and DD in room4
    rooms = get_input(s)
    result = get_min_cost(rooms)
    print(f'Result {rooms}: {result}')
    return result


def get_min_cost(rooms):
    situation = 'N.......NNN' + _concat_rooms(rooms)
    minim_cost = {situation: 0}
    pending_situations = {situation}
    while pending_situations:
        situation = pending_situations.pop()
        moves = get_moves(situation)
        # print(f'{situation} -> {moves}')
        for move in moves:
            new_sit, cost = move
            cur_cost = minim_cost[situation]
            new_cost = cost + cur_cost
            if new_sit not in minim_cost or new_cost < minim_cost[new_sit]:
                minim_cost[new_sit] = new_cost
                pending_situations.add(new_sit)
    print(f'{len(minim_cost)} positions')
    # print([(k, v) for k, v in minim_cost.items() if 'NABCDNNNNNNABCD' in k])
    result = minim_cost['N.......NN' +
                        'NABCDNNNNN' +
                        'NABCDNNNNNN']
    return result


def get_moves(situation):
    possible_moves = []
    for edge, cost in G.items():
        a, b = edge
        if situation[a] == '.' and situation[b] != '.':
            if a < b:
                new_s = situation[:a] + situation[b] + \
                    situation[a+1:b] + situation[a] + situation[b+1:]
            else:
                new_s = situation[:b] + situation[a] + \
                    situation[b+1:a] + situation[b] + situation[a+1:]
            if is_valid_move(b, a, new_s[a]):
                possible_moves.append((new_s, cost * MOVE_COST[new_s[a]]))
    return possible_moves


def is_valid_move(start, end, value):
    if value == 'A' and end in [11, 21]:
        return True
    if value == 'B' and end in [12, 22]:
        return True
    if value == 'C' and end in [13, 23]:
        return True
    if value == 'D' and end in [14, 24]:
        return True
    if start in range(10, 29):
        return True
    if start in range(10) and end in range(10):
        return True
    return False


def _concat_rooms(rooms):
    result = ''
    for i in range(2):
        for room in rooms:
            result += room[i]
        result += 'N' * 6
    return result


assert _concat_rooms(get_input(
    sample_in1)) == 'BCBDNNNNNNADCANNNNNN', _concat_rooms(get_input(sample_in1))
assert _concat_rooms((('A', 'A'), ('B', 'B'), ('C', 'C'),
                      ('D', 'D'))) == 'ABCDNNNNNNABCDNNNNNN'

for k, v in [
    (('N...B...NNNBC.DNNNNNNADCANNNNN', 20), 'N.......NNNBCBDNNNNNNADCANNNNN'),
    (('N..B....NNNBC.DNNNNNNADCANNNNN', 20), 'N...B...NNNBC.DNNNNNNADCANNNNN'),
    (('N..BC...NNNB..DNNNNNNADCANNNNN', 200), 'N..B....NNNBC.DNNNNNNADCANNNNN'),
    (('N..B....NNNB.CDNNNNNNADCANNNNN', 200), 'N..BC...NNNB..DNNNNNNADCANNNNN'),
    (('N..B....NNNBDCDNNNNNNA.CANNNNN', 1000), 'N..B....NNNB.CDNNNNNNADCANNNNN'),
    (('N..BD...NNNB.CDNNNNNNA.CANNNNN', 2000), 'N..B....NNNBDCDNNNNNNA.CANNNNN'),
    (('N...D...NNNBBCDNNNNNNA.CANNNNN', 20), 'N..BD...NNNB.CDNNNNNNA.CANNNNN'),
    (('N...D...NNNB.CDNNNNNNABCANNNNN', 10), 'N...D...NNNBBCDNNNNNNA.CANNNNN'),
    (('N..BD...NNN..CDNNNNNNABCANNNNN', 20), 'N...D...NNNB.CDNNNNNNABCANNNNN'),
    (('N...D...NNN.BCDNNNNNNABCANNNNN', 20), 'N..BD...NNN..CDNNNNNNABCANNNNN'),
    (('N...DD..NNN.BC.NNNNNNABCANNNNN', 2000), 'N...D...NNN.BCDNNNNNNABCANNNNN'),
    (('N...DD..NNN.BCANNNNNNABC.NNNNN', 1), 'N...DD..NNN.BC.NNNNNNABCANNNNN'),
    (('N...DDA.NNN.BC.NNNNNNABC.NNNNN', 2), 'N...DD..NNN.BCANNNNNNABC.NNNNN'),
    (('N...D.A.NNN.BCDNNNNNNABC.NNNNN', 2000), 'N...DDA.NNN.BC.NNNNNNABC.NNNNN'),
    (('N...D.A.NNN.BC.NNNNNNABCDNNNNN', 1000), 'N...D.A.NNN.BCDNNNNNNABC.NNNNN'),
    (('N....DA.NNN.BC.NNNNNNABCDNNNNN', 2000), 'N...D.A.NNN.BC.NNNNNNABCDNNNNN'),
    (('N.....A.NNN.BCDNNNNNNABCDNNNNN', 2000), 'N....DA.NNN.BC.NNNNNNABCDNNNNN'),
    (('N....A..NNN.BCDNNNNNNABCDNNNNN', 2), 'N.....A.NNN.BCDNNNNNNABCDNNNNN'),
    (('N...A...NNN.BCDNNNNNNABCDNNNNN', 2), 'N....A..NNN.BCDNNNNNNABCDNNNNN'),
    (('N..A....NNN.BCDNNNNNNABCDNNNNN', 2), 'N...A...NNN.BCDNNNNNNABCDNNNNN'),
    (('N.......NNNABCDNNNNNNABCDNNNNN', 2), 'N..A....NNN.BCDNNNNNNABCDNNNNN'),
]:
    assert k in get_moves(v), (k, v, get_moves(v))




In [ ]:
if PART0:
    test1 = prob1(sample_in1)
    assert test1 == 12521, test1
    print('ok test')
    prob1(my_in)
else:
    print('not runned part0')

In [ ]:
# Redefine get_moves and is_valid_move for accessing all hallway in one shot
def is_valid_move(start, end, value):
    if value == 'A' and end in [11, 21]:
        return True
    if value == 'B' and end in [12, 22]:
        return True
    if value == 'C' and end in [13, 23]:
        return True
    if value == 'D' and end in [14, 24]:
        return True
    if start in range(10, 29):
        return True
    if start in range(10) and end in range(10):
        return False   # Do not allow hallway transit
    return False


def get_moves(situation):
    possible_moves = []
    #for edge, cost in G.items():
    #    a, b = edge
    #    if situation[a] == '.' and situation[b] != '.':
    #        if a < b:
    #            new_s = situation[:a] + situation[b] + \
    #                situation[a+1:b] + situation[a] + situation[b+1:]
    #        else:
    #            new_s = situation[:b] + situation[a] + \
    #                situation[b+1:a] + situation[b] + situation[a+1:]
    #        if is_valid_move(b, a, new_s[a]):
    #            possible_moves.append((new_s, cost * MOVE_COST[new_s[a]]))
    for edge, val in G2.items():
        cost, intermediates = val
        a, b = edge
        if situation[a] == '.' and situation[b] != '.' and all(
                [situation[x] == '.' for x in intermediates]):
            if a < b:
                new_s = situation[:a] + situation[b] + \
                    situation[a+1:b] + situation[a] + situation[b+1:]
            else:
                new_s = situation[:b] + situation[a] + \
                    situation[b+1:a] + situation[b] + situation[a+1:]
            if is_valid_move(b, a, new_s[a]):
                possible_moves.append((new_s, cost * MOVE_COST[new_s[a]]))

    return possible_moves


for k, v in [
    (('N...B...NNNBC.DNNNNNNADCANNNNN', 20), 'N.......NNNBCBDNNNNNNADCANNNNN'),
    (('N..B....NNNBC.DNNNNNNADCANNNNN', 40), 'N.......NNNBCBDNNNNNNADCANNNNN'),
    (('N..BC...NNNB..DNNNNNNADCANNNNN', 200), 'N..B....NNNBC.DNNNNNNADCANNNNN'),
    (('N..B....NNNB.CDNNNNNNADCANNNNN', 200), 'N..BC...NNNB..DNNNNNNADCANNNNN'),
    (('N..B....NNNBDCDNNNNNNA.CANNNNN', 1000), 'N..B....NNNB.CDNNNNNNADCANNNNN'),
    (('N..BD...NNNB.CDNNNNNNA.CANNNNN', 2000), 'N..B....NNNBDCDNNNNNNA.CANNNNN'),
    (('N...D...NNNBBCDNNNNNNA.CANNNNN', 20), 'N..BD...NNNB.CDNNNNNNA.CANNNNN'),
    (('N...D...NNNB.CDNNNNNNABCANNNNN', 10), 'N...D...NNNBBCDNNNNNNA.CANNNNN'),
    (('N..BD...NNN..CDNNNNNNABCANNNNN', 20), 'N...D...NNNB.CDNNNNNNABCANNNNN'),
    (('N...D...NNN.BCDNNNNNNABCANNNNN', 20), 'N..BD...NNN..CDNNNNNNABCANNNNN'),
    (('N...DD..NNN.BC.NNNNNNABCANNNNN', 2000), 'N...D...NNN.BCDNNNNNNABCANNNNN'),
    (('N...DD..NNN.BCANNNNNNABC.NNNNN', 1), 'N...DD..NNN.BC.NNNNNNABCANNNNN'),
    (('N...DDA.NNN.BC.NNNNNNABC.NNNNN', 2), 'N...DD..NNN.BCANNNNNNABC.NNNNN'),
    (('N...D.A.NNN.BCDNNNNNNABC.NNNNN', 2000), 'N...DDA.NNN.BC.NNNNNNABC.NNNNN'),
    (('N...D.A.NNN.BC.NNNNNNABCDNNNNN', 1000), 'N...D.A.NNN.BCDNNNNNNABC.NNNNN'),
    (('N.....A.NNN.BCDNNNNNNABCDNNNNN', 4000), 'N...D.A.NNN.BC.NNNNNNABCDNNNNN'),
    (('N.......NNNABCDNNNNNNABCDNNNNN', 8), 'N.....A.NNN.BCDNNNNNNABCDNNNNN'),
]:
    assert k in get_moves(v), (k, v, get_moves(v))




In [ ]:
if PART1:
    test1 = prob1(sample_in1)
    assert test1 == 12521, test1
    print('ok test')
    my_res1 = prob1(my_in)
    assert my_res1 > 15261, my_res1
    assert my_res1 == 15299, my_res1
else:
    print('not running part1')


In [ ]:
_G = {  # Valid "edges" and their cost
    (1, 2): 1, (6, 7): 1,
    # (11, 21): 1, (12, 22): 1, (13, 23): 1, (14, 24): 1,
    (2, 11): 2, (3, 11): 2, (2, 3): 2, (3, 12): 2, (4, 12): 2, (3, 4): 2,
    (4, 13): 2, (5, 13): 2, (4, 5): 2, (5, 14): 2, (6, 14): 2, (5, 6): 2,
    # Add edges for 
    # (21, 31): 1, (31, 41): 1,
    # (22, 32): 1, (32, 42): 1,
    # (23, 33): 1, (33, 43):1,
    # (24, 34): 1, (34, 44): 1,
}

G = {edge: cost for _edge, cost in _G.items()
     for edge in [_edge, (_edge[1], _edge[0])]}
G2 = {edge: val for _edge, val in _G2.items()
     for edge in [_edge, (_edge[1], _edge[0])]}
for edge, cost in G.items():
  G2[edge] = (cost, [])

for edge, val in list(G2.items()):
  if edge[0] > 10:
    G2[(edge[0]+10, edge[1])] = (val[0]+1, val[1] + [edge[0]])
    G2[(edge[0]+20, edge[1])] = (val[0]+1, val[1] + [edge[0], edge[0]+10])
    G2[(edge[0]+30, edge[1])] = (val[0]+1, val[1] + [edge[0], edge[0]+10, edge[0]+20])
  if edge[1] > 10:
    G2[(edge[0], 10 + edge[1])] = (val[0]+1, val[1] + [edge[1]])
    G2[(edge[0], 20 + edge[1])] = (val[0]+1, val[1] + [edge[1], edge[1]+10])
    G2[(edge[0], 30 + edge[1])] = (val[0]+1, val[1] + [edge[1], edge[1]+10, edge[1]+20])



def _concat_rooms(rooms):
    result = ''
    for i in range(4):
        for room in rooms:
            result += room[i]
        result += 'N' * 6
    return result

def is_valid_move(start, end, value):
    if value == 'A' and end in [11, 21, 31, 41]:
        return True
    if value == 'B' and end in [12, 22, 32, 42]:
        return True
    if value == 'C' and end in [13, 23, 33, 43]:
        return True
    if value == 'D' and end in [14, 24, 34, 44]:
        return True
    if start in range(10, 49):
        return True
    if start in range(10) and end in range(10):
        return False   # Do not allow hallway transit
    return False

def get_input(s):
    lines = s.strip().split('\n')
    first_line = re.search('#([ABCD])#([ABCD])#([ABCD])#([ABCD])#', lines[2])
    second_line = re.search('#([ABCD])#([ABCD])#([ABCD])#([ABCD])#', lines[3])
    fl = first_line.groups()
    sl = second_line.groups()
    rooms = [(fl[0], 'D', 'D', sl[0]), (fl[1], 'C', 'B', sl[1]),
             (fl[2], 'B', 'A', sl[2]), (fl[3], 'A', 'C', sl[3])]
    return tuple(rooms)

print(get_input(sample_in1))

In [ ]:
CACHE = {}
def get_min_cost(rooms):
    global CACHE
    situation = 'N.......NNN' + _concat_rooms(rooms)
    minim_cost = {situation: 0}
    pending_situations = {situation}
    i = 1
    while pending_situations:
        situation = pending_situations.pop()
        moves = get_moves(situation)
        # print(f'{situation} -> {moves}')
        for move in moves:
            new_sit, cost = move
            cur_cost = minim_cost[situation]
            new_cost = cost + cur_cost
            if new_sit not in minim_cost or new_cost < minim_cost[new_sit]:
                minim_cost[new_sit] = new_cost
                pending_situations.add(new_sit)
        if i % 100000 == 1:
            print(f'{i}: {len(pending_situations)} - {len(minim_cost)}')
        i += 1
    print(f'{len(minim_cost)} positions')
    # print([(k, v) for k, v in minim_cost.items() if 'NABCDNNNNNNABCDNNNNNNABCDNNNNNNABCD' in k])
    CACHE = minim_cost
    result = minim_cost['N.......NN' +
                        'NABCDNNNNN' +
                        'NABCDNNNNN' +
                        'NABCDNNNNN' +
                        'NABCDNNNNNN']
    return result
if PART2:
    test1 = prob1(sample_in1)
    assert test1 == 44169, test1
    prob1(my_in)

In [ ]:
minim_cost = CACHE
print([(k, v) for k, v in minim_cost.items() if 'ABCDNNNNNNABCD' in k])